Install dependencies:

In [30]:
import kagglehub
import pandas as pd
import numpy as np

Download data from Kaggle:

In [31]:
path=kagglehub.dataset_download("prestonrobertson7/league-of-legends-data-9292022")

Read into a Pandas DataFrame:

In [32]:
fileN = "Sep-09-2022_10000matches.csv"
raw =pd.read_csv(path+"/"+fileN)

/tmp/ipykernel_18509/1426500522.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  raw =pd.read_csv(path+"/"+fileN)


**Cleaning the data**
1. Select correct columns
2. Remove ARAMs (this is a random game mode)

In [33]:
pd.DataFrame.head(raw)

,Unnamed: 0,assists,GameID,GameMode,baronKills,bountyLevel,champExperience,champLevel,championName,damageDealtToObjectives,...,totalTimeSpentDead,trueDamageDealtToChampions,trueDamageTaken,turretTakedowns,turretsLost,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
0,0,20,NA1_4430908515,ARAM,0,0,14531,15,Ashe,112,...,74,797,465,1,4,5,0,0,0,False
1,0,3,4035079604,CLASSIC,0,3,15453,16,Sett,18147,...,156,2556,686,4,4,10,0,0,7,True
2,0,19,4035079604,CLASSIC,0,3,13168,15,Trundle,27626,...,82,956,1571,6,4,21,3,0,11,True
3,0,6,4035079604,CLASSIC,0,1,13619,15,Sylas,10629,...,127,600,935,6,4,8,0,0,5,True
4,0,7,4035079604,CLASSIC,1,1,10674,13,KogMaw,13624,...,118,2202,521,3,4,3,1,1,2,True


In [34]:
raw[raw['deaths']==0]

,Unnamed: 0,assists,GameID,GameMode,baronKills,bountyLevel,champExperience,champLevel,championName,damageDealtToObjectives,...,totalTimeSpentDead,trueDamageDealtToChampions,trueDamageTaken,turretTakedowns,turretsLost,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
17,0,0,4035079613,CLASSIC,0,2,7376,11,Nocturne,27955,...,0,131,0,1,0,15,2,3,3,True
20,0,4,4035079613,CLASSIC,0,2,5228,9,Lux,1320,...,0,18,0,0,0,8,1,1,6,True
40,0,10,4035079666,CLASSIC,0,2,7821,11,Brand,3019,...,0,0,160,5,1,53,5,8,22,True
73,0,2,4035079696,CLASSIC,0,7,9828,12,Zed,6606,...,0,254,497,3,0,13,1,1,4,True
128,0,4,4035089567,CLASSIC,0,10,12847,14,Akali,2263,...,0,636,300,1,2,12,0,0,8,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99753,0,3,4035204981,CLASSIC,0,4,9847,12,Malzahar,4637,...,0,638,196,1,1,9,0,1,6,True
99824,0,9,4035205017,CLASSIC,0,6,10342,13,Xayah,8953,...,0,1692,276,5,0,14,1,0,8,True
99870,0,13,4035205077,CLASSIC,0,1,8612,12,Nami,3831,...,0,554,50,4,0,34,4,3,17,True
99889,0,4,4035205099,CLASSIC,0,5,9880,12,Ezreal,5521,...,0,24,66,2,0,19,2,0,10,True


In [35]:
#remove unwanted columns
col_to_copy=['GameID','win','championName','lane','item0','item1','item2','item3','item4','item5','item6','kills','assists','deaths']
only_rel=raw[col_to_copy].copy()

#add a column for KDA, which is a metric we'll use to see how ahead people are. We set the minimum amount of deaths to 1 to avoid a divide by zero error
#this is standard practice in the game amongst stats compilers
only_rel['deaths'].replace(0,1,inplace=True)
only_rel['KDA']=(only_rel['kills']+only_rel['assists'])/only_rel['deaths']

#remove ARAMs; anything with role = 'NONE' is a non-classic gamemode
only_rel=only_rel[only_rel['lane']!='NONE']
pd.DataFrame.head(only_rel)

/tmp/ipykernel_18509/1158143075.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  only_rel['deaths'].replace(0,1,inplace=True)


,GameID,win,championName,lane,item0,item1,item2,item3,item4,item5,item6,kills,assists,deaths,KDA
1,4035079604,True,Sett,JUNGLE,3047,6630,3153,3075,1028,1036,3340,15,3,6,3.00
2,4035079604,True,Trundle,JUNGLE,3075,2031,3044,3047,6632,1037,3340,4,19,4,5.75
3,4035079604,True,Sylas,MIDDLE,4629,3157,6656,3020,1082,1058,3340,18,6,5,4.80
4,4035079604,True,KogMaw,BOTTOM,6672,3085,1043,1055,3006,0,3340,5,7,6,2.00
5,4035079604,True,Nami,MIDDLE,3853,3158,2065,3916,6616,1052,3364,1,24,1,25.00


Drop all champion/role combinations that don't have at least 400 data points

In [36]:
#this just makes sure masking will work properly in the next code block
only_rel.reset_index(inplace=True)
only_rel.set_index(['championName','lane'],inplace=True)

#make a new dataframe with only workable sample sizes
large = only_rel[only_rel.groupby(['championName','lane']).count()>400].dropna().copy()
large=large.astype({'item0': 'int64','item1': 'int64','item2': 'int64','item3': 'int64','item4': 'int64','item5': 'int64','item6': 'int64'})
large.reset_index(inplace=True)
large.drop('index',axis=1,inplace=True)
pd.DataFrame.head(large)

,championName,lane,GameID,win,item0,item1,item2,item3,item4,item5,item6,kills,assists,deaths,KDA
0,Trundle,JUNGLE,4035079604,True,3075,2031,3044,3047,6632,1037,3340,4.0,19.0,4.0,5.750000
1,Sylas,MIDDLE,4035079604,True,4629,3157,6656,3020,1082,1058,3340,18.0,6.0,5.0,4.800000
2,Viego,JUNGLE,4035079604,False,3047,2031,6632,3153,1031,0,3364,5.0,5.0,9.0,1.111111
3,Yasuo,MIDDLE,4035079604,False,6333,3006,6673,3031,3072,0,3340,9.0,2.0,6.0,1.833333
4,Caitlyn,BOTTOM,4035079604,False,1055,6672,3006,3031,0,0,3340,4.0,5.0,9.0,1.000000


In [37]:
#create a list of all item ids for one-hot encoding
i1 = pd.concat([large['item0'],large['item1'],large['item2'],large['item3'],large['item4'],large['item5'],large['item6']])
items = np.unique(i1,0)

#one-hot encode them
items0 = pd.get_dummies(large['item0'],columns=items)
items1= pd.get_dummies(large['item1'],columns=items)
items2= pd.get_dummies(large['item2'],columns=items)
items3= pd.get_dummies(large['item3'],columns=items)
items4= pd.get_dummies(large['item4'],columns=items)
items5= pd.get_dummies(large['item5'],columns=items)
items6= pd.get_dummies(large['item6'],columns=items)
items_encoded = items0.add(items1,fill_value=0)
items_encoded = items_encoded.add(items2,fill_value=0)
items_encoded = items_encoded.add(items3,fill_value=0)
items_encoded = items_encoded.add(items4,fill_value=0)
items_encoded = items_encoded.add(items5,fill_value=0)
items_encoded = items_encoded.add(items6,fill_value=0)
items_encoded = (items_encoded*1).astype('Int8')
pd.DataFrame.head(items_encoded)

,0,1001,1004,1006,1011,1018,1026,1027,1028,1029,...,7014,7015,7016,7017,7018,7019,7020,7021,7022,8001
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
#check to ensure all item slots were built at least once
(items_encoded.sum()==0).sum()==0

np.True_

In [39]:
final = large.join(items_encoded)
pd.DataFrame.head(final)

,championName,lane,GameID,win,item0,item1,item2,item3,item4,item5,...,7014,7015,7016,7017,7018,7019,7020,7021,7022,8001
0,Trundle,JUNGLE,4035079604,True,3075,2031,3044,3047,6632,1037,...,0,0,0,0,0,0,0,0,0,0
1,Sylas,MIDDLE,4035079604,True,4629,3157,6656,3020,1082,1058,...,0,0,0,0,0,0,0,0,0,0
2,Viego,JUNGLE,4035079604,False,3047,2031,6632,3153,1031,0,...,0,0,0,0,0,0,0,0,0,0
3,Yasuo,MIDDLE,4035079604,False,6333,3006,6673,3031,3072,0,...,0,0,0,0,0,0,0,0,0,0
4,Caitlyn,BOTTOM,4035079604,False,1055,6672,3006,3031,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
#check to ensure shape is right
final.shape[0]==large.shape[0]

True

Save as .csv for next notebook

In [41]:
final.to_csv('data/encodedLeagueData.csv', index=False)